In [2]:
#Import Libraries and Packages

import os
import json

# keras imports
from keras.preprocessing.image import img_to_array, load_img
from keras.utils.data_utils import get_file
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing import image
from keras.models import Model
import tensorflow as tf
from keras.models import load_model
from openvino.runtime import Core

import h5py
import numpy as np
import pickle as pk

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from pathlib import Path

In [5]:
# function for openvino model loading 
def OPvino_model_load(model_path):
    """ 
    Return model IR representation and it's output key
    """
    ie = Core()
    model = ie.read_model(model=model_path, weights=Path(model_path).with_suffix(".bin"))
    compiled_model = ie.compile_model(model=model, device_name="CPU")
    output_key = compiled_model.output(0)
    return compiled_model, output_key


In [6]:

second_model_path = "static_openvino/damageORnot/saved_model.xml"
third_model_path = "static_openvino/FRS_vgg/saved_model.xml"
fourth_model_path = "static_openvino/MMS_vgg/saved_model.xml"

Le model vgg19 convertit en IR n'a pas donné de bons résultas après inférence. On utilisera donc le vgg19 simple

In [7]:
#Loading Models for various checks
#first_check, first_output_key = OPvino_model_load(first_model_path)

second_check, second_output_key  =  OPvino_model_load(second_model_path)
third_check , third_output_key = OPvino_model_load(third_model_path)
fourth_check, fourth_output_key = OPvino_model_load(fourth_model_path)

In [8]:
# Loading  valid categories for identifying cars using VGG19
with open('static/cat_counter.pk','rb') as f:
    cat_counter = pk.load(f)

In [9]:
cat_counter

Counter({('n02814533', 'beach_wagon'): 58.054536994954105,
         ('n03770679', 'minivan'): 136.78895090240985,
         ('n02974003', 'car_wheel'): 21.23486805090215,
         ('n03930630', 'pickup'): 62.5587771985447,
         ('n03594945', 'jeep'): 38.84352175355889,
         ('n04285008', 'sports_car'): 119.89040146674961,
         ('n04037443', 'racer'): 52.93018546697567,
         ('n02930766', 'cab'): 20.662175073637627,
         ('n03100240', 'convertible'): 74.37932709977031,
         ('n03459775', 'grille'): 14.798470051027834,
         ('n04461696', 'tow_truck'): 18.27004921430489,
         ('n02701002', 'ambulance'): 5.881439484655857,
         ('n03977966', 'police_van'): 7.475097002694383,
         ('n03769881', 'minibus'): 24.768777273769956,
         ('n03792972', 'mountain_tent'): 0.5388924267608672,
         ('n03670208', 'limousine'): 42.237966875662096,
         ('n03796401', 'moving_van'): 12.216814251849428,
         ('n02965783', 'car_mirror'): 1.50396963302046

In [10]:
# shortlisting top 27 Categories that VGG16 stores for cars (Can be altered for less or more)
cat_list  = [k for k, v in cat_counter.most_common()[:27]]

In [11]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n03930630', 'pickup'),
 ('n02814533', 'beach_wagon'),
 ('n04037443', 'racer'),
 ('n03670208', 'limousine'),
 ('n03594945', 'jeep'),
 ('n03769881', 'minibus'),
 ('n02974003', 'car_wheel'),
 ('n02930766', 'cab'),
 ('n04461696', 'tow_truck'),
 ('n03459775', 'grille'),
 ('n03796401', 'moving_van'),
 ('n04252225', 'snowplow'),
 ('n03977966', 'police_van'),
 ('n02701002', 'ambulance'),
 ('n04065272', 'recreational_vehicle'),
 ('n04467665', 'trailer_truck'),
 ('n03891332', 'parking_meter'),
 ('n03345487', 'fire_engine'),
 ('n03777568', 'Model_T'),
 ('n03791053', 'motor_scooter'),
 ('n02965783', 'car_mirror'),
 ('n03445924', 'golfcart'),
 ('n04252077', 'snowmobile'),
 ('n02704792', 'amphibian')]

First Check - Car or not

In [12]:
CLASS_INDEX = None

In [13]:
CLASS_INDEX_PATH = 'static/imagenet_class_index.json'

In [14]:
def get_predictions(preds,top=5):
    global CLASS_INDEX
    
    #Load the jason file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    #get the results
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [15]:
def prepare_img_224(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [17]:
def car_categories_check(img_224):
    
    #image = np.reshape(image,(1,3,224,224))
    
    #out = model([image])[output_key]
    
    first_check = load_model('static/vgg19.h5')
    print ("Validating that this is a picture of your car...")
    
    out = first_check.predict(img_224)
    top = get_predictions(out, top=5)
    for j in top[0]:
        if j[0:2] in cat_list:
            print ("Car Check Passed!!!")
            print ("\n")
            return True 
    return False 

Second Check - Damaged or not

In [18]:
def car_damage_check(model,output_key, image):
    print ("Validating that damage exists...")
    
    result = model([image])[output_key]

    prediction = int(result[0])
    
    if prediction == 1:
        print ("Validation complete - proceeding to location and severity determination")
        print ("\n")
        return True 
    else:
        print("bla")
        return False

Third Check - Location Assesment

In [19]:
def location_assessment(model,output_key,image):
    print ("Validating the damage area - Front, Rear or Side")
    
    labels = ["Front","Rear","Side"]
    
    result = model([image])[output_key]
    
    result_index = np.argmax(result)

    prediction = labels[result_index]
    
    print ("Your Car is damaged at - " + prediction)
    print ("Location assesment complete")
    print("\n")

Fourth Check - Severity Assesment

In [20]:
def severity_assessment(model,output_key,image):
    print ("Validating the Severity...")
    
    labels = ["Minor","Moderate","Severe"]
    
    result = model([image])[output_key]
    
    result_index = np.argmax(result)

    prediction = labels[result_index]
    
    
    print ("Your Car damage impact is - " + prediction)
    print ("Severity assesment complete")
    print ("\n")
    print ("Thank you")
    

Engine

In [21]:

def engine(img):
    while True:
        try:
            img_224 = prepare_img_224(img)
            g1 = car_categories_check(img_224)

            if g1 is False:
                print ("Are you sure this is a picture of your car? Please submit another picture of the damage.")
                print ("Hint: Try zooming in/out, using a different angle or different lighting.")
                break

            g2 = car_damage_check(second_check,second_output_key, img_224)

            if g2 is False:
                print ("Are you sure that your car is damaged? Please submit another picture of the damage.")
                print ("Hint: Try zooming in/out, using a different angle or different lighting.")
#                 break

            x = location_assessment(third_check,third_output_key,img_224)
            y = severity_assessment(fourth_check,fourth_output_key,img_224)
            break
        except e:
            print(e)
            print ("Image not accessible. Please try again.")
            break

In [22]:
img_path = 'engine/fleur.jpeg'
engine(img_path)

2022-09-13 17:03:25.832515: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-13 17:03:25.832577: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-13 17:03:25.832616: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (liganium-HP): /proc/driver/nvidia/version does not exist
2022-09-13 17:03:25.838099: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-13 17:03:26.469596: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4110417

2022-09-13 17:03:32.950905: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.


Validating that this is a picture of your car...
Are you sure this is a picture of your car? Please submit another picture of the damage.
Hint: Try zooming in/out, using a different angle or different lighting.


In [23]:
img_path = 'engine/image1.jpg'
engine(img_path)

2022-09-13 17:03:35.433019: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.


Validating that this is a picture of your car...
Car Check Passed!!!


Validating that damage exists...
Validation complete - proceeding to location and severity determination


Validating the damage area - Front, Rear or Side
Your Car is damaged at - Side
Location assesment complete


Validating the Severity...
Your Car damage impact is - Severe
Severity assesment complete


Thank you


In [24]:
img_path = 'engine/158.jpg'
engine(img_path)

Validating that this is a picture of your car...
Car Check Passed!!!


Validating that damage exists...
Validation complete - proceeding to location and severity determination


Validating the damage area - Front, Rear or Side
Your Car is damaged at - Front
Location assesment complete


Validating the Severity...
Your Car damage impact is - Moderate
Severity assesment complete


Thank you


In [25]:
img_path = 'engine/7.jpg'
engine(img_path)

Validating that this is a picture of your car...
Car Check Passed!!!


Validating that damage exists...
Validation complete - proceeding to location and severity determination


Validating the damage area - Front, Rear or Side
Your Car is damaged at - Rear
Location assesment complete


Validating the Severity...
Your Car damage impact is - Severe
Severity assesment complete


Thank you


In [26]:
img_path = 'engine/48.jpg'
engine(img_path)

Validating that this is a picture of your car...
Car Check Passed!!!


Validating that damage exists...
Validation complete - proceeding to location and severity determination


Validating the damage area - Front, Rear or Side
Your Car is damaged at - Rear
Location assesment complete


Validating the Severity...
Your Car damage impact is - Minor
Severity assesment complete


Thank you


In [27]:
img_path = 'engine/56.jpg'
engine(img_path)

Validating that this is a picture of your car...
Are you sure this is a picture of your car? Please submit another picture of the damage.
Hint: Try zooming in/out, using a different angle or different lighting.


In [28]:
img_path = 'engine/53.jpg'
engine(img_path)

Validating that this is a picture of your car...
Car Check Passed!!!


Validating that damage exists...
Validation complete - proceeding to location and severity determination


Validating the damage area - Front, Rear or Side
Your Car is damaged at - Front
Location assesment complete


Validating the Severity...
Your Car damage impact is - Severe
Severity assesment complete


Thank you
